In [10]:

"""
SUPER-RSI

The strategy roughly goes like this:

Buy a position when:
    .RSI 1d / 4h / 1h / 30m / 15m <= 30

Close the position when:
    .RSI 1d / 4h / 1h / 30m / 15m >= 70
    .8% fixed stop loss is hit ?? test
"""

import os
from binance.client import Client
import pandas as pd
import datetime
from backtesting import Backtest, Strategy
from backtesting.lib import resample_apply
import sys
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
import time

# %%
# Binance API
api_key = os.environ.get('binance_api')
api_secret = os.environ.get('binance_secret')


# %%
client = Client(api_key, api_secret)

# backtest with 6 years of price data 
#-------------------------------------
today = date.today() 
# print(today)
# today - 4 years - 200 days
# pastdate = today - relativedelta(years=4) - relativedelta(days=200)
pastdate = today - relativedelta(days=30)

# print(pastdate)
# element = datetime.datetime.strptime(str(pastdate_4years),"%Y-%m-%d")
tuple = pastdate.timetuple()
timestamp = time.mktime(tuple)
# print(timestamp)
# dt_object = datetime.datetime.fromtimestamp(timestamp)
# print(dt_object)

startdate = str(timestamp)
# startdate = "15 Dec, 2018 UTC"
# startdate = "12 May, 2022 UTC"
# startdate = "4 year ago UTC"
# startdate = "10 day ago UTC"
#-------------------------------------


# example when want to choose specific start date
#-------------------------------------
# pastdate = datetime.date(2022, 4, 23)
# pastdate = pastdate - relativedelta(days=200)
# tuple = pastdate.timetuple()
# timestamp = time.mktime(tuple)
# startdate = str(timestamp)
#-------------------------------------


# %%

def EMA(values, n):
    """
    Return exp moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).ewm(span=n, adjust=False).mean()

def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()


def RSI(values, n):
    """Relative strength index"""
    # Approximate; good enough
    gain = pd.Series(values).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)


class System(Strategy):
    rsi_length_1d = 10  # RSI lookback periods
    rsi_length_4h = 10
    rsi_length_1h = 10
    rsi_length_30m = 15
    rsi_length_15m = 15
    rsi_low = 20
    rsi_high = 80
    
    def init(self):
        # Compute moving averages the strategy demands
        # self.ma20 = self.I(SMA, self.data.Close, 20)
        # self.SMA50 = self.I(SMA, self.data.Close, 50)
        # self.SMA200 = self.I(SMA, self.data.Close, 200)
        
        
        # Compute RSI
        self.rsi_15m = self.I(RSI, self.data.Close, self.rsi_length_15m)
        self.rsi_30m = resample_apply('30min', RSI, self.data.Close, self.rsi_length_30m)
        self.rsi_1h  = resample_apply('1H', RSI, self.data.Close, self.rsi_length_1h)
        self.rsi_4h  = resample_apply('4H', RSI, self.data.Close, self.rsi_length_4h)
        self.rsi_1d  = resample_apply('D', RSI, self.data.Close, self.rsi_length_1d)
        
        
    def next(self):
        price = self.data.Close[-1]
        # SMA50 = self.SMA50
        # SMA200 = self.SMA200

        # accumulationPhase = (price > SMA50) and (price > SMA200) and (SMA50 < SMA200)
        # bullishPhase = (price > SMA50) and (price > SMA200) and (SMA50 > SMA200)

        # If we don't already have a position, and
        # if all conditions are satisfied, enter long.
        if (not self.position and
                # (accumulationPhase or bullishPhase) and
                self.rsi_15m[-1] <= self.rsi_low and
                self.rsi_30m[-1] <= self.rsi_low and
                self.rsi_1h[-1]  <= self.rsi_low and
                self.rsi_4h[-1]  <= self.rsi_low and
                self.rsi_1d[-1]  <= self.rsi_low):
            self.buy()
        
        # 
        else: 
            if (self.rsi_15m[-1] >= self.rsi_high and
                self.rsi_30m[-1] >= self.rsi_high and
                self.rsi_1h[-1]  >= self.rsi_high and
                self.rsi_4h[-1]  >= self.rsi_high and
                self.rsi_1d[-1]  >= self.rsi_high): 
                self.position.close()
            

# %%
def get_data(Symbol, time_frame, start_date):
    frame = pd.DataFrame(client.get_historical_klines(Symbol,
                                                      time_frame,
                                                      start_date
                                                      ))
    
    frame = frame.iloc[:,:6] # use the first 5 columns
    frame.columns = ['Time','Open','High','Low','Close','Volume'] #rename columns
    frame[['Open','High','Low','Close','Volume']] = frame[['Open','High','Low','Close','Volume']].astype(float) #cast to float
    frame['Date'] = frame['Time'].astype(str) 
    # set the 'date' column as the DataFrame index
    frame.set_index(pd.to_datetime(frame['Date'], unit='ms'), inplace=True) #make human readable timestamp)
    frame = frame.drop(['Date'], axis=1)
    # frame.index = [dt.datetime.fromtimestamp(x/1000.0) for x in frame.Time]


    # format = '%Y-%m-%d %H:%M:%S'
    # frame['Time'] = pd.to_datetime(frame['Time'], format=format)
    
    # frame = frame.set_index(pd.DatetimeIndex(frame['Time']))
    # frame = frame.drop(['Time'], axis=1)
    return frame



In [11]:
symbol = "ETHUSDT"
time_frame = client.KLINE_INTERVAL_15MINUTE
start_date = startdate
df = get_data(symbol, time_frame, start_date)
df

,Time,Open,High,Low,Close,Volume
Date,,,,,,
2023-02-07 00:00:00,1675728000000,1614.28,1619.60,1613.14,1619.38,3871.6199
2023-02-07 00:15:00,1675728900000,1619.38,1621.64,1616.72,1621.51,3669.2136
2023-02-07 00:30:00,1675729800000,1621.51,1621.74,1616.22,1617.28,3948.9621
2023-02-07 00:45:00,1675730700000,1617.27,1621.72,1616.77,1620.23,2529.3906
2023-02-07 01:00:00,1675731600000,1620.24,1625.13,1619.89,1624.82,3538.2889
...,...,...,...,...,...,...
2023-03-09 10:00:00,1678356000000,1532.30,1533.84,1530.36,1533.43,4312.8551
2023-03-09 10:15:00,1678356900000,1533.43,1533.52,1531.66,1532.57,1876.2414
2023-03-09 10:30:00,1678357800000,1532.57,1533.00,1530.12,1530.54,2860.3618


In [3]:
# df_30m = df.resample('30min').last()
# df_30m
# df_1h = df.resample('1H').last()
# df_1h
# df_4h = df.resample('4H').last()
# df_4h
# df_1d = df.resample('1D').last()
# df_1d

In [12]:
%%time
bt = Backtest(df, System, cash=100000, commission=0.001)
stats = bt.run()
print(stats)

Start                     2023-02-07 00:00:00
End                       2023-03-09 11:00:00
Duration                     30 days 11:00:00
Exposure Time [%]                         0.0
Equity Final [$]                     100000.0
Equity Peak [$]                      100000.0
Return [%]                                0.0
Buy & Hold Return [%]               -5.224839
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

In [5]:
# bt.plot()

In [15]:
# bt.optimize(d_rsi=range(10, 35, 5),
#                   w_rsi=range(10, 35, 5),
#                   level=range(30, 80, 10))

stats, heatmap = bt.optimize(
            rsi_length_1d  = range(10, 20, 5),  # RSI lookback periods
            rsi_length_4h  = range(10, 20, 5),
            rsi_length_1h  = range(10, 20, 5),
            rsi_length_30m = range(10, 20, 5),
            rsi_length_15m = range(10, 20, 5),
            rsi_low = range(20, 30, 5),
            rsi_high = range(80, 90, 5),
            maximize='Equity Final [$]',
            return_heatmap=True
            )

In [7]:
# bt.plot(filename='aaa')
# bt.plot()

In [16]:
stats

Start                     2023-02-07 00:00:00
End                       2023-03-09 11:00:00
Duration                     30 days 11:00:00
Exposure Time [%]                         0.0
Equity Final [$]                     100000.0
Equity Peak [$]                      100000.0
Return [%]                                0.0
Buy & Hold Return [%]               -5.224839
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

In [17]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration


In [10]:
heatmap

rsi_length_1d  rsi_length_4h  rsi_length_1h  rsi_length_30m  rsi_length_15m  rsi_low  rsi_high
10             10             10             10              10              20       80          1.077803e+06
                                                                                      85          1.429262e+06
                                                                             25       80          3.952597e+05
                                                                                      85          3.134222e+05
                                                             15              20       80          1.077803e+06
                                                                                                      ...     
15             15             15             15              10              25       85          7.841211e+05
                                                             15              20       80                   NaN
                 

In [18]:
heatmap.dropna()
df_best = heatmap.sort_values(ascending=False).head(1)
df_best

rsi_length_1d  rsi_length_4h  rsi_length_1h  rsi_length_30m  rsi_length_15m  rsi_low  rsi_high
10             10             10             10              10              20       80         NaN
Name: Equity Final [$], dtype: float64

In [28]:
# df_best = pd.DataFrame(df_super_rsi)
rsi_1d = df_best.index.get_level_values(0)[0]
rsi_4h = df_best.index.get_level_values(1)[0]
rsi_1h = df_best.index.get_level_values(2)[0]
rsi_30m = df_best.index.get_level_values(3)[0]
rsi_15m = df_best.index.get_level_values(4)[0]
rsi_low = df_best.index.get_level_values(5)[0]
rsi_high = df_best.index.get_level_values(6)[0]

return_perc = round(stats['Return [%]'],2)
buyhold_return_perc = round(stats['Buy & Hold Return [%]'],2)
backtest_start_date = stats['Start'].strftime('%Y-%m-%d %H:%M:%S')
# backtest_start_date = backtest_start_date.strftime('%Y-%m-%d %H:%M:%S')
# print('a')
# print(type(backtest_start_date))
backtest_end_date = stats['End'].strftime('%Y-%m-%d %H:%M:%S')
num_trades = stats['# Trades']

# lista
print("rsi_1d = ",rsi_1d)
print("rsi_4h = ",rsi_4h)
print("rsi_1h = ",rsi_1h)
print("rsi_30m = ",rsi_30m)
print("rsi_15m = ",rsi_15m)
print("rsi_low = ",rsi_low)
print("rsi_high = ",rsi_high)
print("Return [%] = ",round(return_perc,2))
print("Buy & Hold Return [%] = ",round(buyhold_return_perc,2))
print("Backtest start date =", backtest_start_date)
print("Backtest end date =", backtest_end_date)
print("Trades =", num_trades)

rsi_1d =  10
rsi_4h =  10
rsi_1h =  10
rsi_30m =  10
rsi_15m =  10
rsi_low =  20
rsi_high =  80
Return [%] =  0.0
Buy & Hold Return [%] =  -5.22
Backtest start date = 2023-02-07 00:00:00
Backtest end date = 2023-03-09 11:00:00
Trades = 0


In [33]:
values_list = [symbol, rsi_1d, rsi_4h, rsi_1h, rsi_30m, rsi_15m, rsi_low, rsi_high, return_perc, buyhold_return_perc, backtest_start_date, backtest_end_date, num_trades]
values_list

['ETHUSDT',
 10,
 10,
 10,
 10,
 10,
 20,
 80,
 0.0,
 -5.22,
 '2023-02-07 00:00:00',
 '2023-03-09 11:00:00',
 0]

In [34]:
import pandas as pd
import os

filename = 'best_rsi.csv'
headers = ['SYMBOL', 'RSI_1D', 'RSI_4H', 'RSI_1H', 'RSI_30M', 'RSI_15M', 'RSI_LOW', 'RSI_HIGH', 'RETURN_PERC', 'BUYHOLD_RETURN_PERC', 'BACKTEST_START_DATE', 'BACKTEST_END_DATE', 'NUM_TRADES']
# create a list with the variable values
values_list = [symbol, rsi_1d, rsi_4h, rsi_1h, rsi_30m, rsi_15m, rsi_low, rsi_high, return_perc, buyhold_return_perc, backtest_start_date, backtest_end_date, num_trades]

# check if file exists
if not os.path.isfile(filename):
    # create file and add headers
    pd.DataFrame(columns=headers).to_csv(filename, index=False)

# load csv file into dataframe
df = pd.read_csv(filename)

# check if SYMBOL exists in dataframe
if values_list[0] in df['SYMBOL'].values:
    # update the row
    df.loc[df['SYMBOL'] == values_list[0]] = values_list
else: # SYMBOL not exist
    # append the row
    df = df.append(pd.Series(values_list, index=headers), ignore_index=True)

# save the updated dataframe to csv
df.to_csv(filename, index=False)


In [31]:
import os
import sqlite3

# database
# get the path to the current directory
dir_path = os.getcwd()
# Print the current working directory
# print("Current working directory:", dir_path)
# join the directory path with the name of the database file
db_path = os.path.join(dir_path, "super-rsi.db")
# create a connection to the database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute('''CREATE TABLE IF NOT EXISTS best_rsi
                (symbol TEXT, rsi_1d REAL, rsi_4h REAL, rsi_1h REAL, rsi_30m REAL, 
                 rsi_15m REAL, rsi_low REAL, rsi_high REAL, return_perc REAL, 
                 buyhold_return_perc REAL, backtest_start_date TEXT, backtest_end_date TEXT, 
                 num_trades INTEGER, PRIMARY KEY(symbol))''')

In [38]:

# symbol = 'ETHUSDT'

# rsi_1d = 10
# rsi_4h = 10
# rsi_1h = 10
# rsi_30m = 10
# rsi_15m = 10
# rsi_low = 20
# rsi_high = 80

# return_perc = round(100.10,2)
# buyhold_return_perc = round(200.2,2)
# backtest_start_date = Timestamp('2018-08-19 00:00:00') 
# backtest_end_date = Timestamp('2018-08-19 00:00:00')
# num_trades = 3

# values_list
values_list = [symbol, rsi_1d, rsi_4h, rsi_1h, rsi_30m, rsi_15m, rsi_low, rsi_high, return_perc, buyhold_return_perc, backtest_start_date, backtest_end_date, num_trades]
# insert or update to database

# Check if connection is closed before executing SQL query and committing changes
# Check if connection is closed before committing changes
try:
    cursor.execute('SELECT 1')
except sqlite3.ProgrammingError:
    print("Connection is closed. Opening a new connection.")
    conn = sqlite3.connect('super-rsi.db')
    cursor = conn.cursor()

cursor.execute('INSERT OR REPLACE INTO best_rsi VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', values_list)
conn.commit()
conn.close()

Connection is closed. Opening a new connection.
